In [108]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from art.estimators.classification import KerasClassifier #No soporta TF 2
import os
from PIL import Image

# Disabling eager execution from TF 2
tf.compat.v1.disable_eager_execution()

In [109]:
#Cargamos el modelo

vulnerable_model = tf.keras.models.load_model("modelo_victima_lab6")

In [110]:
# load images as numpy arrays from directory 'malimg_paper_dataset_imgs'
images = []
labels = []
main_directory = './malimg_paper_dataset_imgs'
for subfolder in os.listdir(main_directory):
    subfolder_path = os.path.join(main_directory, subfolder)
    if os.path.isdir(subfolder_path):
        # Iterate over files in the subfolder
        for file_name in os.listdir(subfolder_path):
            # Check if the file has the extension .png
            if file_name.endswith('.png'):
                file_path = os.path.join(subfolder_path, file_name)
                # Load the image using PIL
                image = Image.open(file_path)
                image = image.resize((256,256))
                image = image.convert('L')
                # Convert the image to a NumPy array and add it to the list
                image = np.array(image)
                #if (image_np.shape == (410, 512)):
                images.append([image.tolist()])
                labels.append(subfolder)

In [111]:
# encode categorical labels to integers
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)

images = np.array(images)
labels = np.array(labels)

In [112]:
#Dividir el dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [113]:
# usar el 25% de los datos de entrenamiento
train_images_stolen = X_train
train_labels_stolen = y_train

In [114]:
clasificador = KerasClassifier(
    model=vulnerable_model,
    clip_values=(0, 1))

In [115]:
from art.attacks.extraction import CopycatCNN
# Creating the "neural net thief" object
# that will steal the original classifier
copycat_cnn = CopycatCNN(
    batch_size_fit=256,
    batch_size_query=256,
    nb_epochs=20,
    nb_stolen=len(X_train)*2,
    classifier=clasificador
    )

In [116]:
num_classes = 25

#Definimos un nuevo modelo
def create_blank_model():
    # Defining the model
    model = tf.keras.models.Sequential([
        layers.Rescaling(1./255, input_shape=(256, 256, 1)), # normaliza los valores de los pixeles
        layers.Conv2D(16, 1, padding='same', activation='relu'), # capa convolucional
        layers.MaxPooling2D(), # capa de pooling
        layers.Conv2D(64, 1, padding='same', activation='relu'), # capa convolucional
        layers.MaxPooling2D(), # capa de pooling
        layers.Flatten(), # capa de aplanamiento
        layers.Dense(128, activation='relu'), # capa densa
        layers.Dense(num_classes) # capa densa
    ])

    # Compiling the model
    model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    # Returning the model
    return model

In [117]:
model_stolen = KerasClassifier(
    model=create_blank_model(),
    clip_values=(0, 1)
    )

In [118]:
X_test = X_test.reshape(X_test.shape[0], 256, 256, 1)

In [119]:
# Extracting a thieved classifier
# by training the reference model
stolen_classifier = copycat_cnn.extract(
    x=X_test, 
    y=y_test, 
    thieved_classifier=model_stolen
    )

Train on 1868 samples
Epoch 1/20
1868/1868 [==============================] - 29s 15ms/sample - loss: 11.4068 - accuracy: 0.1874
Epoch 2/20
1868/1868 [==============================] - 25s 13ms/sample - loss: 7.7648 - accuracy: 0.2848
Epoch 3/20
1868/1868 [==============================] - 25s 13ms/sample - loss: 2.9533 - accuracy: 0.3587
Epoch 4/20
1868/1868 [==============================] - 25s 13ms/sample - loss: 1.8525 - accuracy: 0.4625
Epoch 5/20
1868/1868 [==============================] - 24s 13ms/sample - loss: 1.5521 - accuracy: 0.5327
Epoch 6/20
1868/1868 [==============================] - 24s 13ms/sample - loss: 1.2892 - accuracy: 0.6210
Epoch 7/20
1868/1868 [==============================] - 25s 13ms/sample - loss: 1.0495 - accuracy: 0.7559
Epoch 8/20
1868/1868 [==============================] - 26s 14ms/sample - loss: 0.7959 - accuracy: 0.8164
Epoch 9/20
1868/1868 [==============================] - 26s 14ms/sample - loss: 0.5784 - accuracy: 0.8822
Epoch 10/20
1868/1868 [

In [120]:
# Testing the performance of the original classifier
score_original = clasificador._model.evaluate(
    x=X_test, 
    y=y_test
    )

# Testing the performance of the stolen classifier
score_stolen = stolen_classifier._model.evaluate(
    x=X_test, 
    y=y_test
    )

# Comparing test losses
print(f"Original test loss: {score_original[0]:.2f} " 
      f"vs stolen test loss: {score_stolen[0]:.2f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.2f} " 
      f"vs stolen test accuracy: {score_stolen[1]:.2f}")

Original test loss: 0.48 vs stolen test loss: 3.08
Original test accuracy: 0.87 vs stolen test accuracy: 0.86
